In [ ]:
import numpy as np
import os, datetime
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from tqdm import tqdm
from threading import Thread

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [ ]:
PATH_TO_CKPT = '/media/data/LocalizationData/TFObjectDetection/inference/first_stage_ssd_mobilenet2.pb/frozen_inference_graph.pb'
PATH_TO_LABELS = "/media/data/LocalizationData/TFObjectDetection/data/copter_label_map.pbtxt"
NUM_CLASSES = 8

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
PATH_TO_TEST_IMAGES_DIR = '/media/data/LocalizationData/Validation'
TEST_IMAGE_PATHS = []
for root, dirs, files in os.walk(PATH_TO_TEST_IMAGES_DIR):
    for f in files:
        if f.endswith(".jpg"):
            TEST_IMAGE_PATHS.append(root+'/'+f)
TEST_IMAGE_PATHS.sort()

In [ ]:
def create_tensor_dict():
    # Get handles to input and output tensors
    ops = tf.get_default_graph().get_operations()
    all_tensor_names = {output.name for op in ops for output in op.outputs}
    tensor_dict = {}
    for key in [
            'num_detections', 'detection_boxes', 'detection_scores',
            'detection_classes', 'detection_masks'
    ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
            tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                    tensor_name)
    return tensor_dict

def inference(sess, tensor_dict, image):
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
       feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
            'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

def save_visualization(image_np, output_dict,i):
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=2)
    image_pil = Image.fromarray(np.uint8(image_np)).convert('RGB')
    with tf.gfile.Open('/media/data/LocalizationData/Output/Inference/'+TIMESTAMP+'/'+str(i), 'w') as fid:
        image_pil.save(fid, 'PNG')

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

SAVE_RESULTS=False
TIMESTAMP = "{:%Y-%m-%d-%H-%M-%S}".format(datetime.datetime.now())
if SAVE_RESULTS:
    os.makedirs('/media/data/LocalizationData/Output/Inference/'+TIMESTAMP, exist_ok=True)
i = 0
#image = Image.open('/media/data/LocalizationData/Output/Compositor/2018-03-30-14-52-48-imgs/2018-03-30-14-52-48-img0.png').convert('RGB')
#image = image.resize((200,150))
#image_np = load_image_into_numpy_array(image)
#image_np_expanded = np.expand_dims(image_np, axis=0)
with detection_graph.as_default():
    with tf.Session() as sess:
        tensor_dict = create_tensor_dict()
        for image_path in tqdm(TEST_IMAGE_PATHS):
            # It's way faster to resize before TF (about 2.15s vs 3.2s)
            image = Image.open(image_path).convert('RGB').resize((200,150))
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            output_dict = inference(sess,tensor_dict,image_np)
            
            if SAVE_RESULTS:
                thread = Thread(
                    target=save_visualization,
                    args=(image_np, output_dict, i)
                )
                thread.start()            
            i=i+1